# ComCam Fields with Cell-Based Coadds

Created by: Miranda Gorsuch

A few main purposes of this notebook:
- Look at how many warps are in each ComCam field, and see if the sum of each field is equal to the total number of warps in the ComCam repo.
    - This is to understand what fields are available and check if anything is missing.
- Check the number of default coadds in a field compared to the number of cell-based coadds.
    - This is to check what patches fail for cell-based coadds and to understand why by looking at the task logs.
- Visually inspect cell-based coadd outputs

This notebook also acts as an example for accessing cell-based coadds in ComCam.

Last working weekly: `w_2025_12`

Container size: small (4 GB)

## Imports

In [ ]:
from lsst.daf.butler import Butler
from lsst.skymap import Index2D
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

# define image display backend
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('firefly')

In [ ]:
repo = '/repo/main'
collection = 'LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810'
butler = Butler(repo, collections=[collection])
registry = butler.registry

## Count Warps

List the field names found in the ComCam repo.

In [ ]:
# find all field names in repo
target_names = []
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',).expanded():
    target_names.append(ref.dataId.records["visit"].target_name)
target_names = np.unique(target_names)
print(target_names)

### Total Warps

Count the total number of direct warps found in the ComCam repo, regardless of field type.

In [ ]:
count_repo = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1'):
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

Start the counter for the sum of warps in each individual field.

In [ ]:
warp_count = 0

### Individual Fields

#### Field: SV_38_7

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('Rubin_SV_38_7')"):
    count += 1
    warp_count += 1
print("Number of warps in Rubin_SV_38_7: ", count)

#### Field: ECDFS

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('ECDFS')"):
    count += 1
    warp_count += 1
print("Number of warps in ECDFS: ", count)

#### Field: EDFS

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('EDFS_comcam')"):
    count += 1
    warp_count += 1
print("Number of warps in EDFS_ComCam: ", count)

#### Field: SV_95-25

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('Rubin_SV_095_-25')"):
    count += 1
    warp_count += 1
print("Number of warps in Rubin_SV_95_-25: ", count)

#### Field: Fornax

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('Fornax_dSph')"):
    count += 1
    warp_count += 1
print("Number of warps in Fornax: ", count)

#### Field: 47_Tuc

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('47_Tuc')"):
    count += 1
    warp_count += 1
print("Number of warps in 47_Tuc: ", count)

#### Field: Misc

In [ ]:
count = 0
for ref in butler.registry.queryDatasets('deepCoadd_directWarp', 
                                         collections=collection, 
                                         instrument='LSSTComCam', 
                                         band='i', 
                                         skymap='lsst_cells_v1',
                                         where="visit.target_name IN ('slew_icrs')"):
    count += 1
    warp_count += 1
print("Number of warps in slew_icrs: ", count)

### Total warp count from individual fields

In [ ]:
print("Total counts from repo: ", count_repo)
print("Total counts from individual fields within repo: ", warp_count)

## Check if cell-based coadds ran to completion for each field

If a patch fails for a cell-based coadd, look at the log and use visual inspection to understand why.

The cell-based coadds used in the following collections were generated with the `w_2025_07` stack + `DM-46961` ticket branch.

### Method for plotting cell inputs for a patch

In [ ]:
def plot_cell_inputs(cell_coadd, tract, patch):
    '''
    Method for plotting the number of inputs per cell for a cell-based coadd.
    
    Inputs:
    cell_coadd --> MultipleCellCoadd : patch-size cell-based coadd. Pulled from the butler
    tract --> int : tract of patch being plotted
    patch --> int : patch being plotted
    '''
    
    cell_list_filled = list(cell_coadd.cells.keys())
    
    cell_list = []
    cells_filled = [False] * cell_coadd.grid.shape[0] * cell_coadd.grid.shape[1]
    
    index = 0
    for i in range(cell_coadd.grid.shape[0]):
        for j in range(cell_coadd.grid.shape[1]):
            # Index2D axes are reverse of 2D list
            cell_list.append(Index2D(x=j,y=i))
    
            for cell in cell_list_filled:
                if (j==cell.x) and (i==cell.y):
                    cells_filled[index]=True
    
            index += 1
    
    # arrange the number of inputs for each cell into a 2D grid:
    x_cell_bin = np.arange(0,cell_coadd.grid.shape[0]+1,1)
    y_cell_bin = np.arange(0,cell_coadd.grid.shape[1]+1,1)
    
    inputs_list = np.zeros([cell_coadd.grid.shape[0],cell_coadd.grid.shape[1]])
    
    # for cell in cell_list:
    for index, cell in enumerate(cell_list):
    
        # check if the cell is filled, skip if not
        if not (cells_filled[index]):
            continue 
            
        num_cell_inputs = len(cell_coadd.cells[cell].inputs)
        x_index = cell.x
        y_index = cell.y
            
        inputs_list[y_index][x_index] = num_cell_inputs
    
    fig, ax1 = plt.subplots(1, 1, figsize=(6,6))
    plt.tight_layout()
    
    input_distrib = ax1.pcolormesh(inputs_list, shading='auto', vmin=inputs_list.min(), vmax=inputs_list.max())
    ax1.set_title(f"ComCam cell input distribution\nTract {tract}, Patch {patch}")
    ax1.set_xlabel("Cell x-index")
    ax1.set_ylabel("Cell y-index")
    ax1.set_aspect('equal')
    ax1.set_xticks(np.arange(0, 21, 2))
    ax1.set_yticks(np.arange(0, 21, 2))
    fig.colorbar(input_distrib, ax=ax1, cmap='viridis', shrink=0.75, label='Number of input warps')
    
    plt.show()

### Rubin_SV_38_7 Field

In [ ]:
collection = 'u/mgorsuch/ComCam_Cells/Rubin_SV_38_7/20250214T210230Z'
butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
count_repo = 0
coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoadd', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
count_repo = 0
cell_coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoaddCell', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    cell_coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

Use sets to find what `dataIds` are present in default coadds but not cell-based coadds. This is to a quick check to see which, if any, cell-based coadds failed compared to default coadds.

In [ ]:
set(coadd_dataids)-set(cell_coadd_dataids)

In [ ]:
coadd = butler.get('deepCoadd', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 10704, 
                     patch = 27,
                     band = 'i',)

In [ ]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.image)

In [ ]:
coadd_log = butler.get('assembleCellCoadd_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 10704, 
                     patch = 27,
                     band = 'i',)

coadd_error = [log['message'] for log in coadd_log.dict() if log['levelname'] == 'ERROR']
print(coadd_error)

The coadd dropped from the cell-based coadds here is from no warp fully overlapping any cells.

#### Ensure no empty borders

Double check that the cells on the patch border are filled.

In [ ]:
cell_coadd = butler.get('deepCoaddCell', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 10464, 
                     patch = 87,
                     band = 'i',)

plot_cell_inputs(cell_coadd, 10464, 87)

### ECDFS Field

In [ ]:
collection = 'u/mgorsuch/ComCam_Cells/ECDFS/20250217T221024Z'
butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
count_repo = 0
coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoadd', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of default coadds in the collection: ", count_repo)

In [ ]:
count_repo = 0
cell_coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoaddCell', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    cell_coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of cell-based coadds in the collection: ", count_repo)

In [ ]:
set(coadd_dataids)-set(cell_coadd_dataids)

In [ ]:
coadd = butler.get('deepCoadd', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 4848, 
                     patch = 70,
                     band = 'i',)

In [ ]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.image)

In [ ]:
coadd_log = butler.get('assembleCellCoadd_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 5063, 
                     patch = 57,
                     band = 'i',)

coadd_error = [log['message'] for log in coadd_log.dict() if log['levelname'] == 'ERROR']
print(coadd_error)

All but one of the missing cell-based coadds contains the `No CCD from a warp found within a cell` error.

The only exception is one (tract 5063, patch 57) that has `pop from an empty set` error. This specific coadd doesn't have any warps that fully cover a cell, as seen before.

#### Ensure no empty borders

In [ ]:
cell_coadd = butler.get('deepCoaddCell', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 4848, 
                     patch = 72,
                     band = 'i',)

plot_cell_inputs(cell_coadd, 4848, 72)

### EDFS Field

In [ ]:
collection = 'u/mgorsuch/ComCam_Cells/EDFS/20250214T210850Z'
butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
count_repo = 0
coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoadd', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
count_repo = 0
cell_coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoaddCell', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    cell_coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
set(coadd_dataids)-set(cell_coadd_dataids)

In [ ]:
coadd = butler.get('deepCoadd', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 2394, 
                     patch = 63,
                     band = 'i',)

In [ ]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.image)

In [ ]:
coadd_log = butler.get('assembleCellCoadd_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 2394, 
                     patch = 63,
                     band = 'i',)

coadd_error = [log['message'] for log in coadd_log.dict() if log['levelname'] == 'ERROR']
print(coadd_error)

The two failling cell-based coadds don't have enough warps to fully cover any cells. This is not an unexpected result.

#### Ensure no empty borders

In [ ]:
cell_coadd = butler.get('deepCoaddCell', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 2394, 
                     patch = 25,
                     band = 'i',)

plot_cell_inputs(cell_coadd, 2394, 25)

### Rubin_SV_095_-25 Field

In [ ]:
collection = 'u/mgorsuch/ComCam_Cells/Rubin_SV_95_25/20250219T024719Z'
butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
count_repo = 0
coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoadd', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
count_repo = 0
cell_coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoaddCell', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    cell_coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
set(coadd_dataids)-set(cell_coadd_dataids)

In [ ]:
coadd = butler.get('deepCoadd', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 5526, 
                     patch = 67,
                     band = 'i',)

In [ ]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.image)

In [ ]:
coadd_log = butler.get('assembleCellCoadd_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 5526, 
                     patch = 9,
                     band = 'i',)

coadd_error = [log['message'] for log in coadd_log.dict() if log['levelname'] == 'ERROR']
print(coadd_error)

All but two of the missing cell-based coadds contains the `pop from an empty set` error.

The only exceptions are (tract 5525, patch 11) and (tract 5526, patch 9) that has `No CCD from a warp found within a cell` error.

#### Ensure no empty borders

In [ ]:
cell_coadd = butler.get('deepCoaddCell', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 5525, 
                     patch = 33,
                     band = 'i',)

plot_cell_inputs(cell_coadd, 5525, 33)

### Fornax Field

In [ ]:
collection = 'u/mgorsuch/ComCam_Cells/fornax/20250219T025052Z'
butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
count_repo = 0
coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoadd', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
count_repo = 0
cell_coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoaddCell', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    cell_coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
set(coadd_dataids)-set(cell_coadd_dataids)

In [ ]:
coadd = butler.get('deepCoadd', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 4218, 
                     patch = 25,
                     band = 'i',)

In [ ]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.image)

In [ ]:
coadd_log = butler.get('assembleCellCoadd_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 4218, 
                     patch = 25,
                     band = 'i',)

coadd_error = [log['message'] for log in coadd_log.dict() if log['levelname'] == 'ERROR']
print(coadd_error)

All but one of the missing cell-based coadds contains the `pop from an empty set` error.

The exceptions is (tract 4016, patch 72) that has `No CCD from a warp found within a cell` error.

#### Ensure no empty borders

In [ ]:
cell_coadd = butler.get('deepCoaddCell', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 4016, 
                     patch = 50,
                     band = 'i',)

plot_cell_inputs(cell_coadd, 4016, 50)

### 47_tuc Field

In [ ]:
collection = 'u/mgorsuch/ComCam_Cells/47_Tuc/20250219T025226Z'
butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
count_repo = 0
coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoadd', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
count_repo = 0
cell_coadd_dataids = []
for ref in butler.registry.queryDatasets('deepCoaddCell', collections=collection, instrument='LSSTComCam', band='i', skymap='lsst_cells_v1'):
    cell_coadd_dataids.append(ref.dataId)
    count_repo += 1
print("Total number of warps in the collection: ", count_repo)

In [ ]:
set(coadd_dataids)-set(cell_coadd_dataids)

In [ ]:
coadd = butler.get('deepCoadd', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 453, 
                     patch = 56,
                     band = 'i',)

In [ ]:
display1 = afwDisplay.Display(frame=1)
display1.mtv(coadd.image)

In [ ]:
coadd_log = butler.get('assembleCellCoadd_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 453, 
                     patch = 56,
                     band = 'i',)

coadd_error = [log['message'] for log in coadd_log.dict() if log['levelname'] == 'ERROR']
print(coadd_error)

Two of the missing cell-based coadds contains the `pop from an empty set` error.

The only exception is (tract 453, patch 56) that has `No CCD from a warp found within a cell` error.

#### Ensure no empty borders

In [ ]:
cell_coadd = butler.get('deepCoaddCell', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = 453, 
                     patch = 42,
                     band = 'i',)

plot_cell_inputs(cell_coadd, 453, 42)